In [1]:
import sys
sys.path.append("../../")
from taxcalc import *
import pandas as pd
import numpy as np
import copy

In [2]:
CBO_growth = {2013: {'_factor_target': [0.022, 
                                        0.024,
                                        0.024,
                                        0.025,
                                        0.026,
                                        0.023,
                                        0.018,
                                        0.019,
                                        0.021,
                                        0.021,
                                        0.021,
                                        0.02,
                                        0.02,
                                        0.02]
                     }
              }
behavior_traits = {2017: {'_BE_inc': [0.0], '_BE_sub': [0.4], '_BE_cg': [-3.487]}}

# Create the policies and calculators

In [3]:
policy_cl = Policy()
behavior_cl = Behavior()
records_cl = Records("../../puf.csv")
calc_cl = Calculator(policy_cl, records_cl, behavior_traits)
calc_cl.growth.update_economic_growth(CBO_growth)
for i in range(4):
    calc_cl.increment_year()
assert calc_cl.current_year == 2017
calc_cl.calc_all()
print "Done"

You loaded data for 2009.
Your data have been extrapolated to 2013.
Done


In [4]:
policy_amt = Policy()
behavior_amt = Behavior()
records_amt = Records("../../puf.csv")
reform_amt = {
    2017: {
        '_AMT_trt1': [0.0],
        '_AMT_trt2': [0.0]
    }
}
policy_amt.implement_reform(reform_amt)
calc_amt = Calculator(policy_amt, records_amt, behavior_amt)
calc_amt.growth.update_economic_growth(CBO_growth)
for i in range(4):
    calc_amt.increment_year()
assert calc_amt.current_year == 2017
calc_amt.calc_all()
calc_amt.behavior.update_behavior(behavior_traits)
calc_amt_beh = Behavior.response(calc_cl, calc_amt)
print "Done"

You loaded data for 2009.
Your data have been extrapolated to 2013.
Done


In [25]:
policy_bush = Policy()
behavior_bush = Behavior()
records_bush = Records("../../puf.csv")
reform_bush = {
    2017: {
        '_II_rt7': [0.35],
        '_CG_rt3': [0.15],
        '_II_prt': [0.0],
        '_ID_prt': [0.0]
    }
}
policy_bush.implement_reform(reform_bush)
calc_bush = Calculator(policy_bush, records_bush, behavior_bush)
calc_bush.growth.update_economic_growth(CBO_growth)
for i in range(4):
    calc_bush.increment_year()
assert calc_bush.current_year == 2017
calc_bush.calc_all()
calc_bush.behavior.update_behavior(behavior_traits)
calc_bush_beh = Behavior.response(calc_cl, calc_bush)
print "Done"

You loaded data for 2009.
Your data have been extrapolated to 2013.
Done


In [4]:
policy_all = Policy()
behavior_all = Behavior()
records_all = Records("../../puf.csv")
reform_all = {
    2017: {
        '_II_rt1': [0.12],
        '_II_rt2': [0.12],
        '_II_rt3': [0.25],
        '_II_rt4': [0.25],
        '_II_rt5': [0.33],
        '_II_rt6': [0.33],
        '_II_rt7': [0.33],
        '_AMT_trt1': [0.0],
        '_AMT_trt2': [0.0],
        '_STD': [[12000, 24000, 12000, 18000, 24000, 12000, 1050]],
        '_STD_Aged': [[0,0,0,0,0,0],
                      [0,0,0,0,0,0],
                      [0,0,0,0,0,0],
                      [0,0,0,0,0,0]],
        '_II_em': [0,0,0,0],
        '_CTC_ps': [[75000, 150000, 75000, 75000, 75000, 75000]],
        '_II_credit': [[500, 500, 500, 500, 500, 500]],
        '_II_credit_ps': [[75000, 150000, 75000, 75000, 75000, 75000]],
        '_II_credit_prt': [0.05],
        '_ID_StateLocalTax_HC': [1.0],
        '_ID_Medical_HC': [1.0],
        '_ID_Casualty_HC': [1.0],
        '_ID_Miscellaneous_HC': [1.0],
        '_ID_RealEstate_HC': [1.0],
        '_ALD_Investment_ec': [0.5],        
        '_CG_as_II': [1],
        '_NIIT_trt': [0.0],
        '_PT_rt1': [0.12],
        '_PT_rt2': [0.12],
        '_PT_rt3': [0.25],
        '_PT_rt4': [0.25],
        '_PT_rt5': [0.25],
        '_PT_rt6': [0.25],
        '_PT_rt7': [0.25],
    }
}
policy_all.implement_reform(reform_all)
calc_all = Calculator(policy_all, records_all, behavior_all)
calc_all.growth.update_economic_growth(CBO_growth)
for i in range(4):
    calc_all.increment_year()
assert calc_all.current_year == 2017
calc_all.calc_all()
calc_all.behavior.update_behavior(behavior_traits)
calc_all_beh = Behavior.response(calc_cl, calc_all)
print "Done"

You loaded data for 2009.
Your data have been extrapolated to 2013.
Done


# Iterator and revenue impact

In [24]:
def static_path_change(calcx, calcy):
    static_path = [0] * 10
    calc1 = copy.deepcopy(calcx)
    calc2 = copy.deepcopy(calcy)
    for i in range(10):
        calc1.calc_all()
        calc2.calc_all()
        calc1_combined = (calc1.records._combined * calc1.records.s006)
        calc2_combined = (calc2.records._combined * calc2.records.s006)
        static_path[i] = (calc2_combined - calc1_combined).sum() / 10**9
        if calc1.current_year < 2026:
            calc1.increment_year()
            calc2.increment_year()
    return static_path

In [8]:
path1 = static_path_change(calc_cl, calc_all)
print 'Path assuming no behavioral response'
for x in path1:
    print x

Path assuming no behavioral response
-131.843584362
-132.198655917
-130.985800385
-131.678278683
-133.913164122
-136.899484006
-140.901096815
-145.926118821
-151.907940963
-159.267877981


In [9]:
path2 = static_path_change(calc_cl, calc_all_beh)
print 'Path with behavioral response'
for x in path2:
    print x

Path with behavioral response
-72.5617007519
-71.0588341713
-68.2000962959
-66.2603337797
-65.2341484462
-64.6321439775
-64.8104868233
-65.9967283282
-67.9847547448
-71.1858239708


In [10]:
path3 = static_path_change(calc_amt, calc_all)
print 'Path assuming no behavioral response, compared against just AMT repeal'
for x in path3:
    print x

Path assuming no behavioral response, compared against just AMT repeal
-90.0502072348
-87.6839315762
-83.8213174612
-81.5829360926
-80.7560193346
-80.3251794505
-80.8665726192
-82.4002213804
-84.7642609467
-88.3829007171


In [12]:
path4 = static_path_change(calc_amt_beh, calc_all_beh)
print 'Path with behavioral response, compared against just AMT repeal'
for x in path4:
    print x

Path with behavioral response, compared against just AMT repeal
-32.4253917904
-28.2718366115
-22.8144670774
-17.9984388032
-14.0537611256
-10.1639263913
-7.08105847759
-5.04099095517
-3.64693012625
-3.38600160145


In [26]:
path5 = static_path_change(calc_bush, calc_all)
print 'Path assuming no behavioral response, compared against pre-2009 law'
for x in path5:
    print x

Path assuming no behavioral response, compared against pre-2009 law
-81.8368908136
-80.3660078371
-77.6635469778
-76.379298125
-76.2326639694
-76.647068846
-77.5915667533
-79.3603892768
-81.7206836572
-85.0040415736


In [27]:
path6 = static_path_change(calc_bush_beh, calc_all_beh)
print 'Path with behavioral response, compared against pre-2009 law'
for x in path6:
    print x

Path with behavioral response, compared against pre-2009 law
-50.3514650272
-48.3333378884
-45.200405687
-42.7046446834
-40.7756342297
-39.1578447839
-37.9263631349
-37.4881800966
-37.5632347193
-38.4155558429


# Diagnostics

In [7]:
def taxableGain(calc):
    tg = np.maximum(0., np.minimum(calc.records.c23650, calc.records.p23250)) + calc.records.e01100
    return tg

def num_ided(calc):
    ided = calc.records.s006[(calc.records.c04470 > 0.) * (calc.records.c00100 > 0.)].sum()
    return ided
def num_std(calc):
    std = calc.records.s006[(calc.records._standard > 0.) * (calc.records.c00100 > 0.)].sum()
    return std
def wavgMTR_wages(calc):
    mtr = ((calc.mtr('e00200p')[1] * calc.records.e00200 * calc.records.s006).sum() / 
                   (calc.records.e00200 * calc.records.s006).sum())
    return mtr
def wavgMTR_cg(calc):
    gain = taxableGain(calc)
    mtr = ((calc.mtr('p23250')[1] * gain * calc.records.s006).sum() / 
                 (gain * calc.records.s006).sum())
    return mtr
def wavgMTR_int(calc):
    mtr = ((calc.mtr('e00300')[1] * calc.records.c00300 * calc.records.s006).sum() / 
                   (calc.records.c00300 * calc.records.s006).sum())
    return mtr
def wavgMTR_div(calc):
    mtr = ((calc.mtr('e00650')[1] * calc.records.c00650 * calc.records.s006).sum() / 
                   (calc.records.c00650 * calc.records.s006).sum())
    return mtr
def wavgMTR_stcg(calc):
    gain = taxableGain(calc)
    mtr = ((calc.mtr('p22250')[1] * gain * calc.records.s006).sum() / 
                 (gain * calc.records.s006).sum())
    return mtr
def wavgMTR_passthru(calc):
    passthruInc = np.asarray(calc.records.c00900 + calc.records.c26270)
    mtr_c = calc.mtr('e00900p')[1]
    mtr_e = calc.mtr('e02000')[1]    
    wtot = mtr_c * calc.records.c00900 + mtr_e * calc.records.c26270
    mtr = (wtot * calc.records.s006).sum() / (passthruInc * calc.records.s006).sum()
    return mtr
def wavgMTR_sp(calc):
    agi = np.where(calc.records.c00100 < 0., 0., calc.records.c00100)
    mtr = ((calc.mtr('e00900p')[1] * calc.records.c00900 * calc.records.s006).sum() / 
           (calc.records.c00900 * calc.records.s006).sum())
    return mtr
def wavgMTR_schE(calc):
    mtr = ((calc.mtr('e02000')[1] * calc.records.c26270 * calc.records.s006).sum() / 
                   (calc.records.c26270 * calc.records.s006).sum())
    return mtr
def no_inc_tax(calc):
    count = calc.records.s006[calc.records._iitax < 0.0001].sum()
    return count

def lwrMTR_wages(calc1, calc2):
    #calc1 is baseline, calc2 is reform
    diff = calc2.records.s006[(calc2.mtr('e00200p')[2] < calc1.mtr('e00200p')[2])].sum()
    return diff
def lwrMTR_cg(calc1, calc2):
    #calc1 is baseline, calc2 is reform
    diff = calc2.records.s006[(calc2.mtr('p23250')[2] < calc1.mtr('p23250')[2])].sum()
    return diff
def hgrMTR_wages(calc1, calc2):
    #calc1 is baseline, calc2 is reform
    diff = calc2.records.s006[(calc2.mtr('e00200p')[2] > calc1.mtr('e00200p')[2])].sum()
    return diff
def hgrMTR_cg(calc1, calc2):
    #calc1 is baseline, calc2 is reform
    diff = calc2.records.s006[(calc2.mtr('p23250')[2] > calc1.mtr('p23250')[2])].sum()
    return diff
def num_taxcut(calc1, calc2):
    #calc1 is baseline, calc2 is reform
    diff = calc2.records.s006[calc1.records._combined > calc2.records._combined + 0.0001].sum()
    return diff

def diagnose1(calc):
    print str(round(num_ided(calc) / 1000) / 1000.0) + " million taxpayers using itemized deductions"
    print str(round(num_std(calc) / 1000) / 1000.0) + " million taxpayers using the standard deduction"
    print "Average MTR on wage income: " + str(round(wavgMTR_wages(calc) * 10000) / 10000.0)
    print "Average MTR on long-term gains: " + str(round(wavgMTR_cg(calc) * 10000) / 10000.0)
    print "Average MTR on short-term gains: " + str(round(wavgMTR_stcg(calc) * 10000) / 10000.0)
    print "Average MTR on interest: " + str(round(wavgMTR_int(calc) * 10000) / 10000.0)
    print "Average MTR on qualified dividends: " + str(round(wavgMTR_div(calc) * 10000) / 10000.0)
    #print "Average MTR on pass-through income: " + str(round(wavgMTR_passthru(calc) * 10000) / 10000.0)
    #print "Average MTR on sole proprietorships: " + str(round(wavgMTR_sp(calc) * 10000) / 10000.0)
    #print "Average MTR on partnership/S-corp: " + str(round(wavgMTR_schE(calc) * 10000) / 10000.0)
    print "Number paying no income tax: " + str(round(no_inc_tax(calc) / 1000) / 1000.0) + " million"
    
def diagnose2(calc1, calc2):
    #calc1 is baseline, calc2 is reform
    print "Number with lower EMTR on wages: " + str(round(lwrMTR_wages(calc1, calc2) / 1000) / 1000.0) + " million"
    print "Number with higher EMTR on wages: " + str(round(hgrMTR_wages(calc1, calc2) / 1000) / 1000.0) + " million"
    print "Number with lower EMTR on gains: " + str(round(lwrMTR_cg(calc1, calc2) / 1000) / 1000.0) + " million"
    print "Number with higher EMTR on gains: " + str(round(hgrMTR_cg(calc1, calc2) / 1000) / 1000.0) + " million"
    print "Number receiving a tax cut: " + str(round(num_taxcut(calc1, calc2) / 1000) / 1000.0) + " million"

In [8]:
diagnose1(calc_cl)

45.419 million taxpayers using itemized deductions
113.603 million taxpayers using the standard deduction
Average MTR on wage income: 0.2365
Average MTR on long-term gains: 0.222
Average MTR on short-term gains: 0.3136
Average MTR on interest: 0.2378
Average MTR on qualified dividends: 0.1756
Number paying no income tax: 66.972 million


In [9]:
diagnose1(calc_all)

8.553 million taxpayers using itemized deductions
150.383 million taxpayers using the standard deduction
Average MTR on wage income: 0.2221
Average MTR on long-term gains: 0.1499
Average MTR on short-term gains: 0.1499
Average MTR on interest: 0.097
Average MTR on qualified dividends: 0.124
Number paying no income tax: 61.276 million


In [11]:
diagnose1(calc_all_beh)

8.413 million taxpayers using itemized deductions
150.524 million taxpayers using the standard deduction
Average MTR on wage income: 0.2236
Average MTR on long-term gains: 0.1555
Average MTR on short-term gains: 0.1555
Average MTR on interest: 0.115
Average MTR on qualified dividends: 0.125
Number paying no income tax: 60.942 million


In [10]:
diagnose2(calc_cl, calc_all)

Number with lower EMTR on wages: 70.159 million
Number with higher EMTR on wages: 37.489 million
Number with lower EMTR on gains: 51.989 million
Number with higher EMTR on gains: 56.083 million
Number receiving a tax cut: 68.556 million


In [12]:
diagnose2(calc_cl, calc_all_beh)

Number with lower EMTR on wages: 68.979 million
Number with higher EMTR on wages: 35.478 million
Number with lower EMTR on gains: 52.259 million
Number with higher EMTR on gains: 55.997 million
Number receiving a tax cut: 64.834 million


# Average tax rates

In [43]:
def getAGI(calc1):
    return np.asarray(np.where(calc1.records.c00100 < 0, 0., calc1.records.c00100))

def getAvgRate(calc1):
    agi1 = getAGI(calc1) + 0.001
    avgRate1 = np.asarray(calc1.records._combined) / agi1
    avgRate2 = np.where(avgRate1 > 1, 1.0, avgRate1)
    avgRate3 = np.where(avgRate2 < 0, 0., avgRate2)
    return avgRate3

def group_avgRate(calc, bins):
    agi = getAGI(calc)
    rate = getAvgRate(calc)
    wgt = calc.records.s006
    assert bins >= 1 and type(bins)==int
    size = int(219814 / bins)
    
    all_together = zip(agi, rate, wgt)
    all_together.sort()
    all_apart = zip(*all_together)
    agi_sorted = np.asarray(all_apart[0])
    rate_sorted = np.asarray(all_apart[1])
    wgt_sorted = np.asarray(all_apart[2])
    
    avgList = [0] * bins
    wgtList = [0] * bins
    
    for i in range(bins):
        #avgList[i] = (rate_sorted[(i*size):((i+1)*size)] * wgt_sorted[(i*size):((i+1)*size)]).sum()
        #wgtList[i] =  wgt_sorted[(i*size):((i+1)*size)].sum()
        #avgList[i] = np.percentile(rate_sorted[(i*size):((i+1)*size)], 75)
        avgList[i] = np.std(rate_sorted[(i*size):((i+1)*size)])
        wgtList[i] = 1
    
    avgFinal = np.asarray(avgList) / np.asarray(wgtList)
    return avgFinal

def group_avgRate_v2(calc, bins):
    agi = getAGI(calc)
    tax = calc.records._combined
    wgt = calc.records.s006
    ided_ind = np.where(calc_cl.records.s006 != 99999999999, True, False)
    #ided_ind = (calc_xx.records.c04470 > calc_xx.records._standard)
    assert bins >= 1 and type(bins)==int
    size = int(len(tax) / bins)
    
    all_together = zip(agi, tax, wgt, ided_ind)
    all_together.sort()
    all_apart = zip(*all_together)
    agi_sorted = np.asarray(all_apart[0])
    tax_sorted = np.asarray(all_apart[1])
    wgt_sorted = np.asarray(all_apart[2])
    ided_ind_sorted = np.asarray(all_apart[3])
    
    taxList = [0] * bins
    agiList = [0] * bins
    
    for i in range(bins):
        taxList[i] = np.where(ided_ind_sorted[(i*size):((i+1)*size)], tax_sorted[(i*size):((i+1)*size)] * wgt_sorted[(i*size):((i+1)*size)], 0).sum()
        agiList[i] = np.where(ided_ind_sorted[(i*size):((i+1)*size)], agi_sorted[(i*size):((i+1)*size)] * wgt_sorted[(i*size):((i+1)*size)], 0).sum()
        #taxList[i] = np.where(ided_ind_sorted[(i*size):((i+1)*size)], wgt_sorted[(i*size):((i+1)*size)], 0).sum()
        #agiList[i] = 1
        
    avgFinal = np.asarray(taxList) / np.asarray(agiList)
    return avgFinal

In [44]:
xx1 = group_avgRate(calc_cl, 5)
for x in xx1:
    print x
print '\n'
yy1 = group_avgRate(calc_all_beh, 5)
for y in yy1:
    print y
print '\n'
ww1 = group_avgRate(calc_bush_beh, 5)
for z in ww1:
    print z

0.276974353372
0.108690130832
0.0874219958091
0.0902778140939
0.0833352823784


0.300183922959
0.104496801548
0.076300185059
0.0745371420013
0.0614508617176


nan
nan
nan
nan
nan


In [29]:
xx2 = group_avgRate_v2(calc_cl, 5)
for x in xx2:
    print x
print '\n'
yy2 = group_avgRate_v2(calc_all_beh, 5)
for y in yy2:
    print y
print '\n'
zz2 = group_avgRate_v2(calc_bush_beh, 5)
for z in zz2:
    print z

-0.0561916225469
-0.0144714389652
0.0800931549588
0.130250910131
0.262877782476


-0.0513046340257
-0.0153601772687
0.078014847844
0.134463763565
0.247891668729


nan
nan
0.00601237292611
nan
nan
